In [2]:
import requests 
import json 
import tempfile
import pathlib 
from datetime import datetime as dt
from uuid import uuid4
from requests_oauthlib import OAuth1Session
import pandas as pd
import os
import urllib
from os import path

# imports the twitter_secrets python file in which we store the twitter API keys
from twitter_secrets import twitter_secrets as ts



# the write path in which the data will be stored. If it does not yet exist, it will be created
out_path = "/twitter/output/"
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)

# establish the connection by providing the twitter API keys from the twitter file
twitter = OAuth1Session(
    client_key=ts.CONSUMER_KEY,
    client_secret=ts.CONSUMER_SECRET,
    resource_owner_key=ts.ACCESS_TOKEN,
    resource_owner_secret=ts.ACCESS_SECRET)

# 01 Retrieving Tweets by Searchtag

In [3]:
# the max number of tweets that will be returned
max_results = 10

# the hashtag or phrase to fetch the tweets for
searchtag = "2021"

# define the query data
query_data = {
    "track": f"{searchtag}".replace(" ", "").lower(),
    "language": "en", # the language to use
    "date_since": "2019-12-01" # retrieve only tweets after this date
}

# the twitter API url (version 1.1)
url = "https://stream.twitter.com/1.1/statuses/filter.json"

# adds the query data to the url
query_url = f"{url}?{'&'.join([f'{k}={v}' for k, v in query_data.items()])}"
print(query_url)

data = []
print(f"Retrieving max {max_results} Tweets:")
with twitter.get(query_url, stream=True) as response:
    for i, raw_tweet in enumerate(response.iter_lines()):
        if i == max_results:
            break
        if raw_tweet != "b''":     
            try:
                tweet = json.loads(raw_tweet)
                userid = tweet['id']
                user = tweet['user']
                username = user['screen_name']
                userlocation = user['location']
                created_at = tweet['created_at']
                text = tweet['text']
                data.append([userid, username, userlocation, created_at, text])
                #print(f"{i+1}/{max_results}: {user}\n @ {created_at }\n: {text}\n")

            except (json.JSONDecodeError, KeyError) as err:
                # In case the JSON fails to decode, we skip this tweet
                print(f"{i+1}/{max_results}: ERROR: encountered a problem with a line of data... \n")
                continue

            # write the json file to disk
            with pathlib.Path(out_path) / f"{dt.now().timestamp()}_{uuid4()}.json" as F:
                F.write_bytes(raw_tweet)
            
df = pd.DataFrame (data, columns = ['userid', 'username', 'userlocation', 'created_at','text'])
df

https://stream.twitter.com/1.1/statuses/filter.json?track=2021&language=en&date_since=2019-12-01
Retrieving max 10 Tweets:


,userid,username,userlocation,created_at,text
0,1345400537156616194,classicwhiskey,Yavin IV,Sat Jan 02 16:04:19 +0000 2021,"ask yoni\n\nIdk what that means, but okay XD"
1,1345400537164857344,ChucklingBears,None,Sat Jan 02 16:04:19 +0000 2021,Finally! The word is out. So much has been hap...
2,1345400537320095745,Lastone020501,None,Sat Jan 02 16:04:19 +0000 2021,RT @Raze0013: ☀️🐶#MusicBNK48 ✨✨\nHappy New Yea...
3,1345400537248899072,dasGielchen,Germany,Sat Jan 02 16:04:19 +0000 2021,RT @Lindatiny21: ATINY'S VOTE FOR TODAY! Give...
4,1345400537311825923,sherijr,Baltimore MD Citizen 2012,Sat Jan 02 16:04:19 +0000 2021,RT @BoycottUtah: Fellow citizens of Georgia. Y...
5,1345400537345265665,kvetchup,"London, UK to Los Angeles, CA",Sat Jan 02 16:04:19 +0000 2021,@qorquiq @nicholestrano Jacket over game pajam...
6,1345400537328414720,fxyxwxe,🍷•🧀•🍞•🦪•🍋•🧂•🍕•🍟•🍦•🥝,Sat Jan 02 16:04:19 +0000 2021,"RT @asdfghjunyeol: It's 2021, but my heart sti..."
7,1345400537395691523,tarungarg87,None,Sat Jan 02 16:04:19 +0000 2021,RT @FaheemYounus: Should I continue my Twitter...
8,1345400537626402816,BurauFred,Alabama,Sat Jan 02 16:04:19 +0000 2021,"RT @RudyGiuliani: Sunday January 3, 2021, Dr. ..."
9,1345400537584431104,TheBenduPodcast,"Austin, TX",Sat Jan 02 16:04:19 +0000 2021,RT @sw_holocron: REVIEW: Star Wars The High Re...


# 02 Retrieving Tweets from a specific user

In [4]:
# the max number of tweets that will be returned
max_results = 10

# define the query data
screen_name = "Barack Obama"

query_data = {
    "screen_name": f"{screen_name}".replace(" ", "").lower(),
    "date_since": "2019-12-01" # retrieve only tweets after this date
}

# the twitter API url (version 1.1)
url = "https://api.twitter.com/1.1/statuses/user_timeline.json"

# adds the query data to the url
query_url = f"{url}?{'&'.join([f'{k}={v}' for k, v in query_data.items()])}"
print(query_url)

data = []
print(f"Retrieving max {max_results} Tweets:")
with twitter.get(query_url, stream=True) as response:
    for i, raw_tweet in enumerate(response.iter_lines()):
        if i == max_results:
            break
        if raw_tweet != "b''":     
            try:
                tweets = json.loads(raw_tweet)
                for tweet in tweets: 
                    #print(str(tweet))
                    created_at = tweet['created_at']
                    text = tweet['text']
                    retweet_count = tweet['retweet_count']
                    favorite_count = tweet['favorite_count']
                    user = tweet['user']
                    username = user['screen_name']
                    data.append([screen_name, created_at, retweet_count, favorite_count, text])

            except (json.JSONDecodeError, KeyError) as err:
                # In case the JSON fails to decode, we skip this tweet
                print(f"{i+1}/{max_results}: ERROR: encountered a problem with a line of data... \n")
                continue

            # write the json file to disk
            with pathlib.Path(out_path) / f"{dt.now().timestamp()}_{uuid4()}.json" as F:
                F.write_bytes(raw_tweet)
                
df = pd.DataFrame (data, columns = ['screen_name', 'created_at', 'retweet_count', 'favorite_count', 'text'])
df

https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name=barackobama&date_since=2019-12-01
Retrieving max 10 Tweets:


,screen_name,created_at,retweet_count,favorite_count,text
0,Barack Obama,Fri Jan 01 18:25:46 +0000 2021,2244,21721,And here’s a story that reminds us of the powe...
1,Barack Obama,Fri Jan 01 18:25:45 +0000 2021,26121,290527,After a year that has tested us in unimaginabl...
2,Barack Obama,Tue Dec 29 19:11:57 +0000 2020,8443,43484,We’re just one week away from the U.S. Senate ...
3,Barack Obama,Tue Dec 29 17:10:10 +0000 2020,2217,11630,The redistricting process in 2021 will be a sn...
4,Barack Obama,Fri Dec 25 00:01:40 +0000 2020,11809,171442,This Christmas looks different for all of us. ...
5,Barack Obama,Tue Dec 22 18:32:11 +0000 2020,4872,30478,It’s unconscionable that there are families wo...
6,Barack Obama,Mon Dec 21 22:30:31 +0000 2020,23703,230213,"With COVID cases surging worse than ever, gett..."
7,Barack Obama,Sat Dec 19 20:01:04 +0000 2020,33242,286359,Here are some of my favorite songs of the year...
8,Barack Obama,Fri Dec 18 14:01:54 +0000 2020,10629,127740,"Like everyone else, we were stuck inside a lot..."
9,Barack Obama,Thu Dec 17 23:01:50 +0000 2020,2220,17275,"From Willie Mays to Mamie Johnson, the players..."


# 03 Retrieving Images from Twitter

In [5]:
# the max number of tweets that will be returned
max_results = 20

# the hashtag or phrase to fetch the tweets for
searchtag = "chart"

# define the query data
query_data = {
    "track": f"{searchtag}".replace(" ", "").lower(),
    "language": "en", # the language to use" 
    "has": "media"
}
# the twitter API url (version 1.1)
url = "https://stream.twitter.com/1.1/statuses/filter.json"

# adds the query data to the url
query_url = f"{url}?{'&'.join([f'{k}={v}' for k, v in query_data.items()])}"
print(query_url)

def defineFolderName(searchtag):
    # ddmmYY-HMS
    now = dt.now()
    dt_string = now.strftime("%d%m%Y-%H%M%S")
    
    if len(searchtag) > 20:
        return dt_string
    else:
        return dt_string + "-" + searchtag

def createDir(savepath):
    try:
        os.makedirs(savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
        if path.exists(savepath):
            print("path exists")
    else:
        print ("Successfully created the directory %s " % savepath)

def save_images(savepath, tweet, i):
    z = 1
    #print(tweet)
    entities = tweet['entities']
    tweet_url = tweet['id']
    created_at = tweet['created_at']
    tweet_media = entities['media']
    for media in tweet_media: # a tweet can have multiple images/videos
        media_url = str(media['media_url_https'])
        file_name = media['id']
        try:
            pic = urllib.request.urlopen(media_url)
            file_path = savepath + "/" + searchtag + str(i) + "-" + str(z) + ".jpg"
            with open(file_path, 'wb') as localFile:
                localFile.write(pic.read())
            z += 1
            #save image origin info
            data.append([tweet_url, created_at, file_name, media_url])
        except Exception as e:
            print('exception at counter ' + str(counter))
    
# detect the current working directory and print it
base_path = os.getcwd()
print ("The current working directory is %s" % base_path)
img_dir = '/twitter/downloaded_media/'
data = []

# the write path in which the data will be stored. If it does not yet exist, it will be created
file_path = base_path + img_dir

#createDir(file_path)
unique_folder_path = file_path + defineFolderName(searchtag)
createDir(unique_folder_path) 

print(f"Retrieving a total max of {max_results} Tweets:")
with twitter.get(query_url, stream=True) as response:
    for i, raw_tweet in enumerate(response.iter_lines()):
        if i == max_results:
            break
        try:
            tweet = json.loads(raw_tweet)
            save_images(unique_folder_path, tweet, i)
            
        except (json.JSONDecodeError, KeyError) as err:
            # In case the JSON fails to decode, we skip this tweet
            print(f"{i+1}/{max_results}: ERROR: encountered a problem with a line of data... ")
            continue

image_df = pd.DataFrame (data, columns = ['created_at', 'tweet_id', 'media_id', 'media_url'])
image_df

https://stream.twitter.com/1.1/statuses/filter.json?track=chart&language=en&has=media
The current working directory is C:\Users\Flo\relataly-public-python-tutorials
Successfully created the directory C:\Users\Flo\relataly-public-python-tutorials/twitter/downloaded_media/02012021-170430-chart 
Retrieving a total max of 20 Tweets:
{'created_at': 'Sat Jan 02 16:04:26 +0000 2021', 'id': 1345400567498043392, 'id_str': '1345400567498043392', 'text': '@MikeLupica I made a chart going back to 1963 and it showed that out of all the number one qb picks only 3 won the SB.', 'display_text_range': [12, 118], 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': False, 'in_reply_to_status_id': 1345357010552967169, 'in_reply_to_status_id_str': '1345357010552967169', 'in_reply_to_user_id': 4827378145, 'in_reply_to_user_id_str': '4827378145', 'in_reply_to_screen_name': 'MikeLupica', 'user': {'id': 48405814, 'id_str': '48405814', 'name': 'Rusty Austin', 'scree

{'created_at': 'Sat Jan 02 16:04:28 +0000 2021', 'id': 1345400577002356736, 'id_str': '1345400577002356736', 'text': 'RT @TVXQquotes: 8th Years of Catch Me -If you wanna- \n\n"Catch Me" was released by Avex Trax as TVXQ\'s 36th Japan Single in January 2013, de…', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1015626933051142144, 'id_str': '1015626933051142144', 'name': 'irena! 🍋 #HappySungminDay! 🐰', 'screen_name': 'timeIesssorry', 'location': 'sj, tvxq, bb, 2ne1, mmo', 'url': None, 'description': 'here to stan, here to stay | i sometimes tweet about figure skating too | fan acc', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 261, 'friends_count': 801, 'listed_count': 6, 'favourites_count': 84808, 'statuse

{'created_at': 'Sat Jan 02 16:04:28 +0000 2021', 'id': 1345400577694396419, 'id_str': '1345400577694396419', 'text': 'RT @IZONE_stats: [!] @official_izone Panorama | Daily Chart | 210101\n\n#16 Bugs (-1)\n#21 Vibe (-4)\n#50 Genie (=)\n#68 Melon (+7)\n\n#아이즈원', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 727396553242099712, 'id_str': '727396553242099712', 'name': 'One_Reeler', 'screen_name': 'RickOOunix', 'location': 'Bandung, Jawa Barat', 'url': None, 'description': 'Wew', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 41, 'friends_count': 87, 'listed_count': 0, 'favourites_count': 49784, 'statuses_count': 19461, 'created_at': 'Tue May 03 07:17:07 +0000 2016', 'utc_offset': None, 'time_zone': None, 'geo

{'created_at': 'Sat Jan 02 16:04:29 +0000 2021', 'id': 1345400582513766403, 'id_str': '1345400582513766403', 'text': 'RT @MrFelixFixed08: 👇👇\nIndicate on your chart by sir i want to make payment and you will sent and account to make your payment\nIncase any c…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1341006417889792000, 'id_str': '1341006417889792000', 'name': 'MR:FELIX FIXED ODDS', 'screen_name': 'MrFelixFixed08', 'location': None, 'url': None, 'description': 'I deal on correct score fixed matches on daily basis.I maintain 100% winning record,call or whatsApp +2348121734350', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 240, 'friends_count': 1053, 'listed_count': 0, 'favourites_count': 115, 'st

{'created_at': 'Sat Jan 02 16:04:30 +0000 2021', 'id': 1345400584120184832, 'id_str': '1345400584120184832', 'text': 'Boy looked at me, looked at the seating chart, looked at me, then said “what you ticket say?\nTold em 9F, he say, “a… https://t.co/2DoHdOE6of', 'display_text_range': [0, 140], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': True, 'in_reply_to_status_id': 1345399483908153344, 'in_reply_to_status_id_str': '1345399483908153344', 'in_reply_to_user_id': 1142944061004931072, 'in_reply_to_user_id_str': '1142944061004931072', 'in_reply_to_screen_name': 'josheeme', 'user': {'id': 1142944061004931072, 'id_str': '1142944061004931072', 'name': '🖤', 'screen_name': 'josheeme', 'location': 'Kennewick, WA', 'url': None, 'description': '🖤', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 228, 'friends_count': 236, 'listed_count': 0, 'favourites_count': 45, 'statuses_count': 4615, 'created_a

{'created_at': 'Sat Jan 02 16:04:34 +0000 2021', 'id': 1345400599895044100, 'id_str': '1345400599895044100', 'text': 'RT @WONHO_GLOBAL: [HANTEO DATA]\n\nAlbum Chart 📊\n\nLove Synonym Pt.1: Right for Me is the #13 most sold album by a male solo artist.\n\nAlbum Ye…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 48454906, 'id_str': '48454906', 'name': 'Verônica 🇧🇷🏴', 'screen_name': 'vefreiheit94', 'location': 'Brasil', 'url': 'http://instagram.com/vefreiheit94', 'description': '• ARMY 💜 (@BTS_BigHit)\n• WENEE II̷I̷I̷ (@Official__Wonho)\n• MONBEBE ⋈ (@OfficialMonstaX)\n• TO MOON 🌙 (@official_ONEUS)\n• ALIEN 👽 (@TokioHotel)', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 336, 'friends_count': 330, 'listed_c

{'created_at': 'Sat Jan 02 16:04:34 +0000 2021', 'id': 1345400603254554625, 'id_str': '1345400603254554625', 'text': "RT @txtonlive: jan 6: idol wonderland\njan 9: world trigger ost 'force'\njan 9-10: golden disc awards \njan 12: live on yeonjun cameo\njan 13:…", 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1167886309488087041, 'id_str': '1167886309488087041', 'name': 'KaiAD_⁷∞', 'screen_name': 'hwat_is', 'location': None, 'url': None, 'description': '😌🤟', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 17, 'friends_count': 72, 'listed_count': 0, 'favourites_count': 13476, 'statuses_count': 1201, 'created_at': 'Sat Aug 31 19:46:20 +0000 2019', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, '

{'created_at': 'Sat Jan 02 16:04:35 +0000 2021', 'id': 1345400605628514304, 'id_str': '1345400605628514304', 'text': '@hrtblrgmba Chart', 'display_text_range': [12, 17], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': 1345400557578588162, 'in_reply_to_status_id_str': '1345400557578588162', 'in_reply_to_user_id': 1264852172430163968, 'in_reply_to_user_id_str': '1264852172430163968', 'in_reply_to_screen_name': 'hrtblrgmba', 'user': {'id': 1285946799966937090, 'id_str': '1285946799966937090', 'name': 'clawi', 'screen_name': 'ysomeanlol', 'location': 'dehan', 'url': None, 'description': 'yo', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 22, 'friends_count': 25, 'listed_count': 0, 'favourites_count': 96, 'statuses_count': 216, 'created_at': 'Wed Jul 22 14:36:39 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contribut

{'created_at': 'Sat Jan 02 16:04:38 +0000 2021', 'id': 1345400616726609922, 'id_str': '1345400616726609922', 'text': 'RT @talkofthecharts: "Dynamite" by @BTS_twt is projected to remain at #1 on this week\'s Digital Song Sales chart. This will be its 12th wee…', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2830107570, 'id_str': '2830107570', 'name': 'ᴮᴱK✜⁷🧸🍓', 'screen_name': '7_biggerhit', 'location': 'BTSWORLD', 'url': 'https://youtu.be/-5q5mZbe3V8', 'description': 'To that future, running forward with BTS🇧🇹|Fan Account|BE OUT NOW| She/Her', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 411, 'friends_count': 808, 'listed_count': 3, 'favourites_count': 178624, 'statuses_count': 138951, 'created_at': 'Wed S

,created_at,tweet_id,media_id,media_url
0,1345400604244529153,Sat Jan 02 16:04:35 +0000 2021,1345369304632590337,https://pbs.twimg.com/media/Equ2QSZXAAE0jU5.jpg
